In [1]:
import sys
import random
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from pymystem3 import Mystem
from gensim.models import Phrases
from gensim import corpora, models,similarities
import json
import logging
from sklearn.feature_extraction.text import TfidfVectorizer

C:\Users\sanek\Anaconda3\lib\site-packages\gensim\utils.py:1167: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


# Вычисляем tf-idf

In [3]:
random.seed(1228)

В папке data лежат 28 файлов, в каждом файле по 100 текстов на одну тему

In [4]:
event_path = "./data/"

m = Mystem()
regex = re.compile("[А-Яа-яёA-Za-z]+")
mystopwords = stopwords.words('russian') + stopwords.words('english') + stopwords.words('german')
mystopwords += ['это', 'наш', 'т', 'д', 'этот', 'который', 'свой', 'наш', 'мой', 'твой', 'тот', 'делать', 'друг', 'сделать', 
                'любой', 'каждый', 'самый', 'хотеть', 'мочь', 'ваш', 'сей', 'весь', 'очень', 'день', 'год', 'м',
               'ч', 'руб', 'л', 'например', 'становиться', 'http', 'www', 'https', 'vk', 'vkontakte', 'vk', 'com', 'ru',
               'январь', 'февраль','март','апрель','май','июнь','июль','август','сентябрь','октябрь','ноябрь','декабрь']  


Опреляем все необходимые функции:

In [5]:
def remove_url(text):
    text =  re.sub(r"http\S+", "", text).strip()
    text =  re.sub(r"vk\S+", "", text).strip()
    text =  re.sub(r"https\S+", "", text).strip()
    text =  re.sub(r"VK\S+", "", text).strip()
    return text

def words_only(text, regex=regex):
    return " ".join(regex.findall(text))

def remove_stopwords(text, mystopwords = mystopwords):
    try:
        return " ".join([token for token in text.split() if (token not in mystopwords) and (token.count(token[0]) != len(token))])
    except:
        return ""

In [6]:
def lemmatize(text, mystem=m):
    try:
        return "".join(m.lemmatize(text)).strip()  
    except:
        return ""

def length_filter(text, n_words = 3):
    tokens = text.split()
    if len(tokens) > n_words:
        return text
    else:
        return ""

def add_bigrams(texts, min_count = 5): # запись: "дональд_трамп"
    tokens = [text.split() for text in texts]
    bigram = Phrases(tokens, min_count = min_count)
    for i in range(len(tokens)):
        for token in bigram[tokens[i]]:
            if '_' in token:
                tokens[i].append(token)
    return [' '.join(text) for text in tokens]


In [14]:
def news2df(): # разделение текстов из файла, создание data frame
    data = []
    for i in range(1,29):
        with open(event_path + str(i) + '.txt', encoding='utf8') as infile:
            lines = infile.readlines()
        texts = []
        text = ''
        for j in range(13, len(lines)):
            if lines[j] != '\n':
                text += ' '+lines[j].strip()
            else:
                texts.append(text)
                text = ''
            if j == len(lines)-1:
                texts.append(text)
        for text in texts:
            data.append([i, text])
    text_df = pd.DataFrame.from_records(data, columns = ['event_id','text'])  
    return text_df

Вся предобработка входных данных:

In [15]:
def preprocess(df):
    df.text = df.text.str.lower()
    df.text = df.text.apply(remove_url)
    df.text = df.text.apply(words_only)
    df.text.replace('', np.nan, inplace=True)
    df.dropna(subset=['text'], inplace=True)
    df.text = df.text.apply(remove_stopwords)
    df.text = df.text.apply(lemmatize)
    df.text = df.text.apply(remove_stopwords)  
    df.text = add_bigrams(df.text)
    return df

В events2017.xlsx записаны темы текстов

In [16]:
converters = {0: str, 1:str, 2:str}
event_df = pd.read_excel(event_path+"events2017.xlsx",  header=None, sheetname = 0, converters=converters)

In [17]:
event_df = event_df.rename(index=str, columns={0: 'id', 1: "date", 2: "name"})


In [18]:
event_df.head() # 28 строк с названиями тем

,id,date,name
0,1,2017-01-10 00:00:00,Власти Петербурга согласились передать РПЦ Иса...
1,2,2017-01-20 00:00:00,Дональд Трамп вступил в должность президента США.
2,3,2017-02-20 00:00:00,Скоропостижно скончался постпред России при ОО...
3,4,2017-03-02 00:00:00,Вышел фильм Навального «он Вам не димон»
4,5,2017-03-14 00:00:00,CNN показала фильм «Владимир Путин — самый вли...


In [19]:
event_df.to_csv('./events.csv', index = False)

Разделяем и обрабатываем тексты:

In [20]:
news_df = news2df()
news_df.head()
news_df = preprocess(news_df)
news_df.to_csv('./news.csv')


news_df = pd.read_csv('./news.csv',
                     usecols = ['event_id','text'],
                     dtype = {'event_id':'str', 'text':'str'})
event_df = pd.read_csv('./events.csv',
                     usecols = ['id', 'date', 'name'],
                     dtype = {'id':'str', 'date':'str', 'name':'str'})


C:\Users\sanek\Anaconda3\lib\site-packages\gensim\models\phrases.py:486: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xef in position 0: invalid continuation byte

Получили по 100 строк на каждую из 28 тем

In [13]:
news_df

,event_id,text
0,1,петербург проходить митинг против передача иса...
1,1,lenta co москва ситуация передача исаакиевский...
2,1,аргумент факт aif москва оппозиция проводить м...
3,1,google новость топ москва акция против передач...
4,1,газета москва московский патриархия прокоммент...
5,1,top real estate topre москва петербуржец проте...
6,1,findnews москва полиция насчитывать участник а...
7,1,правда info pravda info москва ленинградец соб...
8,1,голос америка golos ameriki москва исаакиевски...
9,1,выбор народ vybor naroda org москва петербург ...


Внутри леммы и биграммы

In [18]:
news_df.text[112] 

'правдинформ trueinform москва трамп вступать должность президент сша дональд трамп инаугурация сша риа новость избранный президент дональд трамп прибывать инаугурация капитолий вашингтон вашингтон янва риа новость дональд трамп вступать должность президент сша приводить присяга председатель верховный суд сша джон робертс бизнесмен трамп президент сша сменить пост глава государство барак обама завершать срок полномочие дональд трамп торжественно клясться добросовестно выполнять обязанность президент соединять штат полный мера сила поддерживать охранять защищать конституция соединять штат произносить трамп подымать правый рука полагать левый рука библия первый библия подарить мать честь окончание воскресный школа второй принадлежать му президент сша авраам линкольн инаугурация глава государство принимать участие праздничный обед сенатор конгрессмен затем принимать торжественный парад вместе первый леди мелания трамп посещать несколько бал ria world html правдинформ_trueinform вступать_д

Объединяем все тексты одной темы

In [19]:
N_event = len(event_df)

merged_news = []
gb = news_df.groupby('event_id')    
for event in range(N_event):
    text = ''
    sample = gb.get_group(str(event+1))
    text = ' '.join(sample.text.tolist())
    merged_news.append(text)

Вычисляем tfidf

In [20]:
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1,1), min_df = 0)
tfidf_matrix =  tfidf.fit_transform(merged_news)
feature_names = tfidf.get_feature_names() 
data = []
dense = tfidf_matrix.todense()
for i in range(N_event):
    text = dense[i].tolist()[0]
    phrase_scores = [pair for pair in zip(range(0, len(text)), text) if pair[1] > 0]
    sorted_phrase_scores = sorted(phrase_scores, key=lambda t: t[1] * -1)
    phrases = []
    for phrase, score in [(feature_names[word_id], score) for (word_id, score) in sorted_phrase_scores][:50]:
        phrases.append(phrase)
    data.append([i, ' '.join(phrases), event_df.name.tolist()[i]])
df = pd.DataFrame.from_records(data, columns = ['event_id','terms','event_name'])
df.to_csv('./event_dict.csv', index = False)

Результат:

In [21]:
for i in range (0, 28):
    print (i)
    print (df.event_name[i])
    print (df.terms[i])

0
Власти Петербурга согласились передать РПЦ Исаакиевский собор.
собор исаакиевский рпц передача исаакиевский_собор передача_исаакиевский петербург музей собор_рпц церковь музейный санкт безвозмездный передача_собор православный акция резник пользование против вишневский против_передача храм буров полтавченко депутат санкт_петербург православный_церковь исаакий максим_резник алексей_ковалев борис_вишневский ковалев встреча_депутат русский_православный безвозмездный_пользование религиозный передавать губернатор исаакиевский_площадь митинг николай_буров решение собираться проходить русский москва человек встреча памятник георгий
1
Дональд Трамп вступил в должность президента США.
инаугурация трамп президент дональд сша дональд_трамп президент_сша обама россия избранный_президент избранный инаугурация_трамп первый российский новый время церемония страна республиканец инаугурационный барак цб мир правительство человек демократ инаугурация_дональд суд вашингтон кабинет назначение американск